In [ ]:

from roboflow import Roboflow

rf = Roboflow(api_key="unauthorized")
project = rf.workspace("buyumedatasets").project("acne-new")
version = project.version(35)
dataset = version.download("yolov5")

In [4]:
from glob import glob

img_list = glob('/home/addinedu/dev_ws/data/Diplom-4/test/images/*.jpg')
print(len(img_list))

307


훈련,테스트 데이터 분할

In [2]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.1, random_state=2000)
print(len(train_img_list), len(val_img_list))

5109 568


In [3]:
with open('/home/addinedu/dev_ws/data/Diplom-4/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/home/addinedu/dev_ws/data/Diplom-4/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')


In [5]:
import yaml

with open('/home/addinedu/dev_ws/data/Diplom-4/data.yaml', 'r') as f:
    data = yaml.safe_load(f)

print(data)

data['train'] = '/home/addinedu/dev_ws/data/Diplom-4/train.txt'
data['val'] = '/home/addinedu/dev_ws/data/Diplom-4/val.txt'


with open('/home/addinedu/dev_ws/data/Diplom-4/data.yaml', 'w') as f:
    yaml.dump(data,f)

print(data)

{'names': ['rokok'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'diplom-grweq', 'url': 'https://universe.roboflow.com/mejnar/diplom-grweq/dataset/4', 'version': 4, 'workspace': 'mejnar'}, 'test': '../test/images', 'train': 'Diplom-4/train/images', 'val': 'Diplom-4/valid/images'}
{'names': ['rokok'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'diplom-grweq', 'url': 'https://universe.roboflow.com/mejnar/diplom-grweq/dataset/4', 'version': 4, 'workspace': 'mejnar'}, 'test': '../test/images', 'train': '/home/addinedu/dev_ws/data/Diplom-4/train.txt', 'val': '/home/addinedu/dev_ws/data/Diplom-4/val.txt'}


모델 생성

In [1]:
%cd /home/addinedu/yolo/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /home/addinedu/dev_ws/data/Diplom-4/data.yaml --cfg /home/addinedu/yolo/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name cigar_yolov5s_3


/home/addinedu/yolo/yolov5


/home/addinedu/venv/YOLO_venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


train: weights=yolov5s.pt, cfg=/home/addinedu/yolo/yolov5/models/yolov5s.yaml, data=/home/addinedu/dev_ws/data/Diplom-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=cigar_yolov5s_3, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 18 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)



In [6]:
import torch
torch.cuda.get_device_name(0)	#gpu 확인
torch.cuda.is_available()

True

/home/addinedu/yolo/yolov5/runs/train/trouble_result2/weights/best.pt

생성한 모델을 가지고 테스트 해보기

In [11]:
from IPython.display import Image
import os

# val_img_path = '/home/addinedu/dev_ws/data/asd/trouble/images.jpeg'
val_img_path = val_img_list[1]
%cd /home/addinedu/yolo/yolov5/
!python detect.py --weights /home/addinedu/yolo/yolov5/runs/train/cigar_yolov5s_2/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"


/home/addinedu/yolo/yolov5
detect: weights=['/home/addinedu/yolo/yolov5/runs/train/cigar_yolov5s_2/weights/best.pt'], source=/home/addinedu/dev_ws/data/Diplom-4/train/images/smoke_1869_jpg.rf.20ac88353008d465b8c6f4d78d3d12a8.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /home/addinedu/dev_ws/data/Diplom-4/train/images/smoke_1869_jpg.rf.20ac88353008d465b8c6f4d78d3d12a8.jpg: 416x416 2 rokoks, 5.2ms
Speed: 0.6ms pre-process,

텍스트 제거, 라인두께 얇게

In [2]:
import os

val_img_path = '/home/addinedu/dev_ws/data/asd/trouble/maxresdefault.jpg'
yolov5_dir = '/home/addinedu/yolo/yolov5'

os.chdir(yolov5_dir)
command = f"python detect.py --weights /home/addinedu/yolo/yolov5/runs/train/trouble_result2/weights/best.pt --img 416 --conf 0.5 --source '{val_img_path}' --hide-labels --line-thickness 1"
os.system(command)

detect: weights=['/home/addinedu/yolo/yolov5/runs/train/trouble_result5/weights/best.pt'], source=/home/addinedu/dev_ws/data/asd/trouble/maxresdefault.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=True, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20873139 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /home/addinedu/dev_ws/data/asd/trouble/maxresdefault.jpg: 256x416 (no detections), 42.1ms
Speed: 1.0ms pre-process, 42.1ms inference, 7.4ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp17


0

In [1]:
%cd /home/addinedu/yolo/yolov5/

!python train.py --img 416 --batch 16 --epochs 20 --data /home/addinedu/dev_ws/data/acne-new.v35i.yolov5pytorch/dataset/data.yaml --cfg /home/addinedu/yolo/yolov5/models/yolov5m.yaml --weights yolov5s.pt --name trouble_result5


/home/addinedu/yolo/yolov5


/home/addinedu/venv/YOLO_venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


train: weights=yolov5s.pt, cfg=/home/addinedu/yolo/yolov5/models/yolov5m.yaml, data=/home/addinedu/dev_ws/data/acne-new.v35i.yolov5pytorch/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=trouble_result5, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

hyperparameters: lr0=0.01, lrf=0.01,

In [6]:
%cd /home/addinedu/yolo/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /home/addinedu/dev_ws/data/yolo_data/data.yaml --cfg /home/addinedu/yolo/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name rock_paper_yolo5


/home/addinedu/yolo/yolov5
train: weights=yolov5s.pt, cfg=/home/addinedu/yolo/yolov5/models/yolov5s.yaml, data=/home/addinedu/dev_ws/data/yolo_data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=rock_paper_yolo5, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

hyperparameters: lr0=0.01, lrf=0.0